In [1]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import numpy as np
import shared
import json


ve_or_ju = "ve"

In [2]:
history = pd.read_csv(f'data/grouped_paces_{ve_or_ju}.tsv', delimiter="\t")
history["first_name"] = history.name.str.split(" ", expand=True).iloc[:, 0]
history.head()

,mean_team_id,teams,name,num_runs,num_valid_times,mean_pace,stdev,log_stdev,most_common_leg,most_common_country,pace_1,pace_2,pace_3,pace_4,pace_5,pace_6,pace_7,pace_8,first_name
0,14.5,STORA TUNA OK,anna mårsell,4,4,6.6108,0.3563,0.0526,1,SWE,6.441,7.214,6.291,6.497,NaN,NaN,NaN,NaN,anna
1,27.0,IFK MORAS OK;STORA TUNA OK,magdalena olsson,7,7,7.1139,0.5656,0.0824,2,SWE,6.511,7.381,7.676,7.710,6.062,7.234,7.223,NaN,magdalena
2,3.0,STORA TUNA OK,julia jakob,1,1,5.9440,0.0000,0.0000,3,SWE,5.944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,julia
3,10.0,STORA TUNA OK,tove alexandersson,7,7,6.0373,0.3487,0.0565,4,SWE,5.711,6.042,6.289,6.008,5.672,6.738,5.801,NaN,tove
4,61.0,EKSJÖ SOK;GÖTEBORG MAJORNA OK,lisa holer,4,3,7.3283,0.8452,0.1119,4,SWE,6.567,6.911,8.507,NaN,NaN,NaN,NaN,NaN,lisa


In [3]:
with_history = history[history["num_valid_times"] >= 1]
names = with_history[['first_name', "mean_pace"]].groupby('first_name').agg(["median", "mean", "std"]).dropna()
names.columns = ['_'.join(col).strip() for col in names.columns.values]
names

,mean_pace_median,mean_pace_mean,mean_pace_std
first_name,,,
,12.39400,12.480800,2.487867
aada,9.30900,9.886200,1.658331
adele,9.50500,9.475933,0.451103
adolfsson,10.15400,10.154000,0.762261
agata,9.55575,9.555750,0.923128
...,...,...,...
åkermarck,13.41700,13.037000,0.913348
åsa,9.41500,10.112592,2.052567
åse,11.34900,12.144333,2.530542


In [4]:
names.shape

(975, 3)

In [5]:
X = names[["mean_pace_median", "mean_pace_mean"]].values
pace_kmeans = KMeans(n_clusters=10, random_state=2019).fit(X)

In [6]:
X_std = names[["mean_pace_std"]].values
pace_std_kmeans = KMeans(n_clusters=4, random_state=2019).fit(X_std)

In [7]:
names = names.assign(fn_pace_class=pace_kmeans.labels_)
names = names.assign(fn_pace_std_class=pace_std_kmeans.labels_)
names.head(10)

,mean_pace_median,mean_pace_mean,mean_pace_std,fn_pace_class,fn_pace_std_class
first_name,,,,,
,12.39400,12.480800,2.487867,0,1
aada,9.30900,9.886200,1.658331,1,1
adele,9.50500,9.475933,0.451103,1,3
adolfsson,10.15400,10.154000,0.762261,7,3
agata,9.55575,9.555750,0.923128,1,3
agne,8.30275,8.302750,0.286732,4,3
agnes,8.30050,8.749800,1.628492,4,1
agnese,9.62000,9.701429,1.621681,1,1
agneta,9.48300,9.706060,1.126797,1,3


In [8]:
names.sort_values(by=["mean_pace_median"])

,mean_pace_median,mean_pace_mean,mean_pace_std,fn_pace_class,fn_pace_std_class
first_name,,,,,
dana,6.4020,6.3610,0.720376,8,3
anastacia,6.4610,6.4610,0.967322,8,3
paulina,6.6060,7.9050,2.300346,8,1
nicole,6.6685,6.6685,0.224153,8,3
miri,6.6939,6.6939,0.289772,8,3
...,...,...,...,...,...
hintikka,17.3740,17.3740,6.310221,2,0
pipsa,17.5515,17.5515,11.711810,2,0
leea,18.0905,18.0905,4.403154,2,2


In [9]:
names.columns = names.columns.get_level_values(0)
#names['first_name'] = names.index
names

,mean_pace_median,mean_pace_mean,mean_pace_std,fn_pace_class,fn_pace_std_class
first_name,,,,,
,12.39400,12.480800,2.487867,0,1
aada,9.30900,9.886200,1.658331,1,1
adele,9.50500,9.475933,0.451103,1,3
adolfsson,10.15400,10.154000,0.762261,7,3
agata,9.55575,9.555750,0.923128,1,3
...,...,...,...,...,...
åkermarck,13.41700,13.037000,0.913348,6,3
åsa,9.41500,10.112592,2.052567,1,1
åse,11.34900,12.144333,2.530542,0,1


In [10]:
names[["fn_pace_class", "fn_pace_std_class"]].to_csv(f"data/name_pace_classes_{ve_or_ju}.tsv", "\t")